In [4]:
import glob

In [5]:
import numpy as np

In [6]:
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.preprocessing.image import random_rotation, random_shift, random_zoom
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.models import Sequential
from keras.models import model_from_json
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils

In [7]:
DirNames = ["./dataset/bad_hair", "./dataset/good_hair"] 
FileNames = ["./dataset/bad_hair.npy", "./dataset/good_hair.npy"]
ClassNames = ["bad_hair", "good_hair"]
hw = {"height":16, "width":16}        #画像サイズ　リストではなく辞書型 中かっこで囲む

In [8]:
def PreProcess(dirname, filename, var_amount=100): #var_amount:一枚の画像から得られるサンプル数
    #定義
    num = 0
    arrlist = []
    #ファイルの読み込み
    files = glob.glob(dirname + "/*.jpg")
    #画像処理
    for imgfile in files:
        img = load_img(imgfile, target_size=(hw["height"], hw["width"]))    # 画像ファイルの読み込み
        array = img_to_array(img) / 255                                     # 画像ファイルのnumpy化
        arrlist.append(array)                 # numpy型データをリストに追加
        for i in range(var_amount-1): # 1枚あたり2枚の回転画像が得られる
            arr2 = array
            arr2 = random_rotation(arr2, rg=360)
            arrlist.append(arr2)              # numpy型データをリストに追加
        num += 1
    nplist = np.array(arrlist)
    np.save(filename, nplist)
    print(">> " + dirname + "から" + str(num) + "個のファイル読み込み成功")

In [9]:
PreProcess(DirNames[0], DirNames[0])
PreProcess(DirNames[1], DirNames[1])
print(np.load("./dataset/bad_hair.npy").shape)

>> ./dataset/bad_hairから1個のファイル読み込み成功
>> ./dataset/good_hairから1個のファイル読み込み成功
(100, 16, 16, 3)


In [10]:
def BuildCNN(ipshape=(32, 32, 3), num_classes=2):
    model = Sequential()
  
     #層1
    model.add(Conv2D(24, 3, padding='same', input_shape=ipshape))
    model.add(Activation('relu'))
  
    #層2
    model.add(Conv2D(48, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
  
    #層3,4
    model.add(Conv2D(96, 3, padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(96, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    #層5
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    #層6
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
  
    #構築
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])
    return model

In [11]:
def Learning(tsnum=30, nb_epoch=50, batch_size=8, learn_schedule=0.9):
    X_TRAIN_list = []; Y_TRAIN_list = []; X_TEST_list = []; Y_TEST_list = [];
    target = 0
    for filename in FileNames :
        data = np.load(filename)          # 画像のnumpyデータを読み込み
        trnum = data.shape[0] - tsnum
        X_TRAIN_list += [data[i] for i in range(trnum)]          # 画像データ
        Y_TRAIN_list += [target] * trnum                         # 分類番号
        X_TEST_list  += [data[i] for i in range(trnum, trnum+tsnum)]          # 学習しない画像データ
        Y_TEST_list  += [target] * tsnum;                                     # 学習しない分類番号
        target += 1
    X_TRAIN = np.array(X_TRAIN_list + X_TEST_list)    # 連結
    Y_TRAIN = np.array(Y_TRAIN_list + Y_TEST_list)    # 連結
    print(">> 学習サンプル数 : ", X_TRAIN.shape)
    y_train = np_utils.to_categorical(Y_TRAIN, target)    # 自然数をベクトルに変換
    valrate = tsnum * target * 1.0 / X_TRAIN.shape[0]

    # 学習率の変更
    class Schedule(object):
        def __init__(self, init=0.001):      # 初期値定義
            self.init = init
        def __call__(self, epoch):           # 現在値計算
            lr = self.init
            for i in range(1, epoch+1):
                lr *= learn_schedule
            return lr
  
    def get_schedule_func(init):
        return Schedule(init)

    lrs = LearningRateScheduler(get_schedule_func(0.001))
    mcp = ModelCheckpoint(filepath='./keras/best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    model = BuildCNN(ipshape=(X_TRAIN.shape[1], X_TRAIN.shape[2], X_TRAIN.shape[3]), num_classes=target)

    print(">> 学習開始")
    hist = model.fit(X_TRAIN, y_train,
                     batch_size=batch_size,
                     verbose=1,
                     epochs=nb_epoch,
                     validation_split=valrate,
                     callbacks=[lrs, mcp])

    json_string = model.to_json()
    json_string += '##########' + str(ClassNames)
    open('./keras/model.json', 'w').write(json_string)
    model.save_weights('./keras/last.hdf5')

In [14]:
Learning(tsnum=30, nb_epoch=50, batch_size=8, learn_schedule=0.9)

>> 学習サンプル数 :  (200, 16, 16, 3)
>> 学習開始


/opt/anaconda3/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
15/18 [========================>.....] - ETA: 0s - loss: 0.7187 - accuracy: 0.5333
Epoch 00001: val_loss improved from inf to 0.67588, saving model to ./keras/best.hdf5
18/18 [==============================] - 1s 29ms/step - loss: 0.7059 - accuracy: 0.5714 - val_loss: 0.6759 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/50
14/18 [======================>.......] - ETA: 0s - loss: 0.6859 - accuracy: 0.5982
Epoch 00002: val_loss improved from 0.67588 to 0.66538, saving model to ./keras/best.hdf5
18/18 [==============================] - 0s 18ms/step - loss: 0.6906 - accuracy: 0.5571 - val_loss: 0.6654 - val_accuracy: 0.5000 - lr: 9.0000e-04
Epoch 3/50
14/18 [======================>.......] - ETA: 0s - loss: 0.6819 - accuracy: 0.5446
Epoch 00003: val_loss improved from 0.66538 to 0.65052, saving model to ./keras/best.hdf5
18/18 [==============================] - 0s 18ms/step - loss: 0.6766 - accuracy: 0.5500 - val_loss: 0.6505 - val_accuracy: 0.5000 - lr: 8.1000e-04
Epoch 4/50
17/1

Epoch 26/50
15/18 [========================>.....] - ETA: 0s - loss: 0.0025 - accuracy: 1.0000
Epoch 00026: val_loss did not improve from 0.00020
18/18 [==============================] - 0s 15ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 2.2289e-04 - val_accuracy: 1.0000 - lr: 7.1790e-05
Epoch 27/50
14/18 [======================>.......] - ETA: 0s - loss: 0.0012 - accuracy: 1.0000
Epoch 00027: val_loss did not improve from 0.00020
18/18 [==============================] - 0s 15ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.0669e-04 - val_accuracy: 1.0000 - lr: 6.4611e-05
Epoch 28/50
14/18 [======================>.......] - ETA: 0s - loss: 0.0028 - accuracy: 1.0000
Epoch 00028: val_loss improved from 0.00020 to 0.00017, saving model to ./keras/best.hdf5
18/18 [==============================] - 0s 18ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 1.7459e-04 - val_accuracy: 1.0000 - lr: 5.8150e-05
Epoch 29/50
16/18 [=========================>....] - ETA: 0s - loss: 0

In [18]:
def TestProcess(imgname):
    modelname_text = open("./keras/model.json").read()
    json_strings = modelname_text.split('##########')
    textlist = json_strings[1].replace("[", "").replace("]", "").replace("\'", "").split()
    model = model_from_json(json_strings[0])
    model.load_weights("./keras/last.hdf5")  # best.hdf5 で損失最小のパラメータを使用
    img = load_img(imgname, target_size=(hw["height"], hw["width"]))    
    TEST = img_to_array(img) / 255

    pred = model.predict(np.array([TEST]), batch_size=1, verbose=0)
    print(">> 計算結果↓\n" + str(pred))
    print(">> この画像は「" + textlist[np.argmax(pred)].replace(",", "") + "」です。")
    print(textlist[np.argmax(pred)] == "good_hair")

In [19]:
TestProcess("./dataset/good_hair/0.jpg")

>> 計算結果↓
[[9.219777e-06 9.999908e-01]]
>> この画像は「good_hair」です。
True


In [1]:
textlist[np.argmax(pred)]

NameError: name 'textlist' is not defined